<a href="https://colab.research.google.com/github/rohit-447/Py-Torch-Learning/blob/main/CNN_MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Loding Modules
import torch
from torch import nn
import torchvision
from torchvision.datasets import FashionMNIST
from torch.utils.data import DataLoader
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt
torch.__version__

In [ ]:
#@title DataLoader

#Downloading and transforming data into tensor
#Train Data
train_data=FashionMNIST(
    root='data',
    download=True,
    transform=ToTensor(),
    train=True
)

#Test Data
test_data=FashionMNIST(
    root='data',
    download=True,
    transform=ToTensor(),
    train=False
)

print(f"\nTrain Data:{len(train_data)}\nTest Data:{len(test_data)}")


In [ ]:
#@title Visulisation of Data

#getting the class of the data
class_idx=FashionMNIST.classes

#displaying a single iamge
img,label=train_data[0]
plt.imshow(img.squeeze(), cmap="grey")
plt.title(class_idx[label])
plt.axis(False)

#displying the multiple images
random_idx=torch.randint(0,len(train_data),size=[1])
plt.figure(figsize=(8,8))
rows=2
cols=5
for i in range(1,rows*cols+1):
  img,label=train_data[i]
  plt.subplot(rows,cols,i)
  plt.imshow(img.squeeze(), cmap="grey")
  plt.title(class_idx[label])
  plt.axis(False)

print(f"Shape of the image:{img.shape}")

In [ ]:
#@title Device Agonistic Code
device="cuda" if torch.cuda.is_available() else "cpu"
print(f'You are on device: {device}')

In [ ]:
#@title DataLoader

#from torch.utils.data import DataLoader
BATCH_SIZE=32
train_dataloader=DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True)
test_dataloader=DataLoader(test_data, batch_size=BATCH_SIZE, shuffle=True)

In [ ]:
#@title Visulaisation of data in dataloader
plt.figure(figsize=(10,10))
rows=8
cols=4
# Get a batch of images and labels
images, labels = next(iter(train_dataloader))

for i in range(rows*cols):
  plt.subplot(rows, cols, i+1)
  plt.imshow(images[i].squeeze(), cmap="grey")
  plt.title(class_idx[labels[i]])
  plt.axis(False)

plt.tight_layout()
plt.show()

In [ ]:
#@title Model using CNN